# Save predictions to Pandas

This notebook loads all 2x5 trained models, the two datasets (fast&slow), gets the predictions and saves the results to a file

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [3]:
import os
# do we have the correct env loaded?
assert 'donkey2' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [4]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/

In [5]:
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False

DIRECTION = 'CC'
N_FOLDS = 5
DATA_PATH = 'data'


________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v4.3.5 ...
loading config file: mycar/config.py
loading personal config over-rides from myconfig.py


In [6]:
from dataloader import load_records
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive/catalog_20.catalog



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/faster-autodrive']


19250

In [7]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/slower-autodrive/catalog_20.catalog


20304

In [8]:
from donkeycar.utils import get_model_by_type

model_type = '3d-modified-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
2022-07-17 23:17:30.267392: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_in (InputLayer)         [(None, 3, 60, 160, 3)]   0         
                                                                 
 conv3d (Conv3D)             (None, 1, 20, 53, 16)     1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 1, 10, 26, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 1, 10, 26, 32)     4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 5, 13, 32)     0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 1, 5, 13, 32)      9248  

In [9]:
from losses import get_ground_truth
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


seq size 3
len ground_truth 19250
seq size 3
len ground_truth 20304


In [11]:
from dataloader import get_fold_indices
from losses import get_predictions
from model_loader import load_model
import numpy as np


train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices'

for fold, (train_index, test_index) in enumerate(get_fold_indices(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


120/120 [==============================] - 61s 505ms/step
predictions shape (15292,)
31/31 [==============================] - 42s 1s/step
predictions shape (3958,)
159/159 [==============================] - 192s 1s/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (20304,)
120/120 [==============================] - 31s 255ms/step
predictions shape (15292,)
31/31 [==============================] - 7s 234ms/step
predictions shape (3958,)
159/159 [==============================] - 36s 224ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (20304,)
121/121 [==============================] - 28s 229ms/step
predictions shape (15472,)
30/30 [==============================] - 6s 210ms/step
predictions shape (3778,)
159/159 [==============================] - 35s 218ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (20304,)
121/121 [==============================] - 27s 223ms/step
predictions shape (15472,)
30/30 [==============================] - 7s 218ms/step
predictions shape (3778,)
159/159 [==============================] - 36s 223ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (20304,)
121/121 [==============================] - 28s 227ms/step
predictions shape (15472,)
30/30 [==============================] - 7s 222ms/step
predictions shape (3778,)
159/159 [==============================] - 35s 221ms/step
predictions shape (20304,)


In [12]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.2505801434457883, -0.2527056067175679, -0.2393154258884921)

In [13]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((19250,), (19250,), (20304,))

In [14]:
np.save('open-loop-results/nighttime-fast-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.save('open-loop-results/nighttime-fast-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy', np.array([slow_predictions, slow_ground_truths]).T)


In [15]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices'

for fold, (train_index, test_index) in enumerate(get_fold_indices(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-1.fold-of-5-CC.h5


127/127 [==============================] - 31s 244ms/step
predictions shape (16129,)
33/33 [==============================] - 8s 245ms/step
predictions shape (4175,)
151/151 [==============================] - 37s 242ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-2.fold-of-5-CC.h5


predictions shape (19250,)
127/127 [==============================] - 31s 241ms/step
predictions shape (16130,)
33/33 [==============================] - 7s 225ms/step
predictions shape (4174,)
151/151 [==============================] - 33s 221ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-3.fold-of-5-CC.h5


predictions shape (19250,)
128/128 [==============================] - 28s 220ms/step
predictions shape (16319,)
32/32 [==============================] - 7s 221ms/step
predictions shape (3985,)
151/151 [==============================] - 34s 225ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-4.fold-of-5-CC.h5


predictions shape (19250,)
128/128 [==============================] - 30s 236ms/step
predictions shape (16319,)
32/32 [==============================] - 7s 226ms/step
predictions shape (3985,)
151/151 [==============================] - 37s 245ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-3d-get-fold-indices/Keras3D_CNN_ModifiedOnlySteering-5.fold-of-5-CC.h5


predictions shape (19250,)
128/128 [==============================] - 31s 241ms/step
predictions shape (16319,)
32/32 [==============================] - 8s 250ms/step
predictions shape (3985,)
151/151 [==============================] - 34s 226ms/step
predictions shape (19250,)


In [16]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.24456062894937428, -0.2491590826787015, -0.25424844947983694)

In [17]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((20304,), (20304,), (19250,))

In [18]:
np.save('open-loop-results/nighttime-slow-auto-3d-model-nighttime-slow-auto-data-prediction-and-ground-truth.npy', np.array([train_predictions, test_predictions, slow_ground_truths]).T)
np.save('open-loop-results/nighttime-slow-auto-3d-model-nighttime-fast-auto-data-prediction-and-ground-truth.npy', np.array([fast_predictions, fast_ground_truths]).T)


In [18]:
cfg.SEQUENCE_LENGTH = 0
cfg.ROI_CROP_TOP = 59


In [19]:
model_type = 'linear-steering'
model = get_model_by_type(model_type, cfg)
model.interpreter.model.summary()


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter


Model: "linear"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img_in (InputLayer)            [(None, 61, 160, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 29, 78, 24)   1824        ['img_in[0][0]']                 
                                                                                                  
 dropout_22 (Dropout)           (None, 29, 78, 24)   0           ['conv2d_1[0][0]']               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 13, 37, 32)   19232       ['dropout_22[0][0]']        

In [20]:
fast_speed_data = load_records('faster-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(fast_speed_data)

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/faster-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/faster-autodrive/catalog_20.catalog


19266

In [21]:
slow_speed_data = load_records('slower-autodrive', cfg, DATA_PATH, TUBRECORD_SIZE)
len(slow_speed_data)


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/slower-autodrive']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/slower-autodrive/catalog_20.catalog


20308

In [22]:
fast_ground_truths = get_ground_truth(model, fast_speed_data)
slow_ground_truths = get_ground_truth(model, slow_speed_data)


len ground_truth 19266
len ground_truth 20308


In [23]:

train_predictions = np.zeros(len(fast_speed_data))
test_predictions = np.zeros(len(fast_speed_data))
slow_predictions = np.zeros(len(slow_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear'

for fold, (train_index, test_index) in enumerate(kf.split(fast_speed_data), start=1):
    train_records = [fast_speed_data[i] for i in train_index]
    test_records = [fast_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(fast_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_slow_predictions = get_predictions(trained_model, cfg, slow_speed_data)
    slow_predictions += fold_slow_predictions[:,0]

train_predictions /= (N_FOLDS-1)
slow_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5


121/121 [==============================] - 93s 767ms/step
predictions shape (15412, 1)
31/31 [==============================] - 12s 382ms/step
predictions shape (3854, 1)
159/159 [==============================] - 96s 603ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5


predictions shape (20308, 1)
121/121 [==============================] - 12s 97ms/step
predictions shape (15413, 1)
31/31 [==============================] - 3s 96ms/step
predictions shape (3853, 1)
159/159 [==============================] - 15s 97ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-3.fold-of-5-CC.h5


predictions shape (20308, 1)
121/121 [==============================] - 12s 98ms/step
predictions shape (15413, 1)
31/31 [==============================] - 3s 96ms/step
predictions shape (3853, 1)
159/159 [==============================] - 15s 97ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5


predictions shape (20308, 1)
121/121 [==============================] - 12s 96ms/step
predictions shape (15413, 1)
31/31 [==============================] - 3s 96ms/step
predictions shape (3853, 1)
159/159 [==============================] - 15s 96ms/step
predictions shape (20308, 1)


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-fast-auto-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5


121/121 [==============================] - 12s 102ms/step
predictions shape (15413, 1)
31/31 [==============================] - 3s 110ms/step
predictions shape (3853, 1)
159/159 [==============================] - 18s 113ms/step
predictions shape (20308, 1)


In [24]:
train_predictions.mean(), test_predictions.mean(), slow_predictions.mean()


(-0.2543695506249286, -0.2543950245587963, -0.24445792576494396)

In [25]:
train_predictions.shape, test_predictions.shape, slow_predictions.shape


((19266,), (19266,), (20308,))

In [26]:
np.savetxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, fast_ground_truths]).T)
np.savetxt('open-loop-results/nighttime-fast-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz', np.array([slow_predictions, slow_ground_truths]).T)


In [27]:

train_predictions = np.zeros(len(slow_speed_data))
test_predictions = np.zeros(len(slow_speed_data))
fast_predictions = np.zeros(len(fast_speed_data))

MODELS_PATH = '/Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear'

for fold, (train_index, test_index) in enumerate(kf.split(slow_speed_data), start=1):
    train_records = [slow_speed_data[i] for i in train_index]
    test_records = [slow_speed_data[i] for i in test_index]
    trained_model = load_model(MODELS_PATH, model_type, cfg, DIRECTION, N_FOLDS, fold)
    fold_train_predictions = np.zeros(len(slow_speed_data))
    np.put(fold_train_predictions, train_index, get_predictions(trained_model, cfg, train_records))
    train_predictions += fold_train_predictions

    fold_test_predictions = get_predictions(trained_model, cfg, test_records)
    np.put(test_predictions, test_index, fold_test_predictions)

    fold_fast_predictions = get_predictions(trained_model, cfg, fast_speed_data)
    fast_predictions += fold_fast_predictions[:,0]

train_predictions /= (N_FOLDS-1)
fast_predictions /= N_FOLDS

INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-1.fold-of-5-CC.h5


127/127 [==============================] - 14s 106ms/step
predictions shape (16246, 1)
32/32 [==============================] - 3s 98ms/step
predictions shape (4062, 1)
151/151 [==============================] - 15s 102ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-2.fold-of-5-CC.h5


predictions shape (19266, 1)
127/127 [==============================] - 12s 97ms/step
predictions shape (16246, 1)
32/32 [==============================] - 3s 98ms/step
predictions shape (4062, 1)
151/151 [==============================] - 14s 95ms/step
predictions shape (19266, 1)


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-3.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-3.fold-of-5-CC.h5


127/127 [==============================] - 13s 98ms/step
predictions shape (16246, 1)
32/32 [==============================] - 3s 95ms/step
predictions shape (4062, 1)
151/151 [==============================] - 15s 98ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-4.fold-of-5-CC.h5


predictions shape (19266, 1)
127/127 [==============================] - 12s 96ms/step
predictions shape (16247, 1)
32/32 [==============================] - 3s 100ms/step
predictions shape (4061, 1)
151/151 [==============================] - 15s 98ms/step


INFO:donkeycar.utils:get_model_by_type: model type is: linear-steering
INFO:donkeycar.parts.keras:Created KerasLinearOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5
INFO:donkeycar.parts.interpreter:Loading model /Volumes/GoogleDrive/My Drive/maka/open-loop/models/nighttime-slow-auto-linear/KerasLinearOnlySteering-5.fold-of-5-CC.h5


predictions shape (19266, 1)
127/127 [==============================] - 12s 97ms/step
predictions shape (16247, 1)
32/32 [==============================] - 3s 101ms/step
predictions shape (4061, 1)
151/151 [==============================] - 15s 101ms/step
predictions shape (19266, 1)


In [28]:
train_predictions.mean(), test_predictions.mean(), fast_predictions.mean()


(-0.24557717313088576, -0.24545020701456136, -0.2573492209899343)

In [29]:
train_predictions.shape, test_predictions.shape, fast_predictions.shape


((20308,), (20308,), (19266,))

In [30]:
np.savetxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-slow-auto-data-prediction-and-ground-truth.csv.gz', np.array([train_predictions, test_predictions, slow_ground_truths]).T)
np.savetxt('open-loop-results/nighttime-slow-auto-linear-model-nighttime-fast-auto-data-prediction-and-ground-truth.csv.gz', np.array([fast_predictions, fast_ground_truths]).T)
